In [1]:
from glob import glob
from astroquery.mast import Mast, Observations
import os
import shutil
import numpy as np
import json
import requests
import asdf
from astropy import log
from astropy.io import ascii, fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table

from align_to_catalogs import *
import astropy.units as u
from reproject.mosaicking import find_optimal_celestial_wcs

fatal: detected dubious ownership in repository at '/blue/adamginsburg/adamginsburg/repos/dask'
To add an exception for this directory, call:

	git config --global --add safe.directory /blue/adamginsburg/adamginsburg/repos/dask


In [2]:
import jwst

print(jwst.__version__)
JP-3285: Allow negative angles in engineering format. Add support for angle units by @mcara in #7683


SyntaxError: illegal target for annotation (1589681051.py, line 4)

In [2]:
#realign_to_catalog(reference_coordinates, filtername='f212n',
#                       module='nrca',
#                       basepath='/orange/adamginsburg/jwst/brick/',
#                       fieldnumber='001',
#                       max_offset=0.4*u.arcsec,
#                       mag_limit=15,
#                       catfile=None, imfile=None)
#
#

In [3]:
0 + 1*u.arcsec

<Quantity 1. arcsec>

In [4]:
fov_regname = {'brick': 'regions/nircam_brick_fov.reg',
              'cloudc': 'regions/nircam_cloudc_fov.reg',
              }
regionname = 'cloudc'
basepath = f'/orange/adamginsburg/jwst/{regionname}/'
filtername = 'F405N'
field = '002' 
module = 'merged'
output_dir = f'/orange/adamginsburg/jwst/{regionname}/{filtername}/pipeline/'

In [5]:
imfile = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak.fits'
imfile_realigned = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak_realigned_to_vvv.fits'
shutil.copy(imfile, imfile_realigned)

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak_realigned_to_vvv.fits'

In [6]:
reference_catalog = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-F405N-merged_vvvcat.fits'

In [7]:
reftbl = Table.read(reference_catalog)

In [8]:
reference_coordinates = reftbl['skycoord']

In [9]:
reftbl['skycoord']

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    [(266.581129, -28.666482), (266.594517, -28.666434),
     (266.604305, -28.663877), ..., (266.555803, -28.513283),
     (266.560494, -28.514834), (266.559398, -28.51401 )]>

In [10]:
cat_file = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv'
cat = Table.read(cat_file)

In [15]:
#cat['xcentroid'], cat['ycentroid']

In [12]:
ww =  WCS(fits.getheader(imfile, ext=('SCI', 1)))
ww

Set DATE-AVG to '2023-04-25T04:14:32.839' from MJD-AVG.
Set DATE-END to '2023-04-25T04:14:59.681' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to   -11.485975 from OBSGEO-[XYZ].
Set OBSGEO-H to 1386082407.725 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 266.59237008461  -28.557172894513  
CRPIX : 1024.5  1024.5  
CD1_1 CD1_2  : -1.6126880967505e-06  -1.7440453913269e-05  
CD2_1 CD2_2  : -1.7358691201541e-05  1.6484453417233e-06  
NAXIS : 2048  2048

In [13]:
skycrds_cat = ww.pixel_to_world(cat['xcentroid'], cat['ycentroid'])
skycrds_cat

<SkyCoord (ICRS): (ra, dec) in deg
    [(266.6116505 , -28.56882209), (266.61113913, -28.57424765),
     (266.61090323, -28.57649018), ..., (266.54867726, -28.65584137),
     (266.54533313, -28.67274956), (266.54349879, -28.68148105)]>

In [16]:
ww.wcs.crval

array([266.59237008, -28.55717289])

In [17]:
#ww.wcs.crval = ww.wcs.crval - [(0.1*u.arcsec).to(u.deg).value, (0.1*u.arcsec).to(u.deg).value]
#ww.wcs.crval

In [18]:
#realign_to_catalog(reference_coordinates=reference_coordinates, filtername=filtername.lower(), module=module, 
#                   basepath=basepath, fieldnumber=field, imfile=imfile_realigned, decoffset=-8*u.arcsec)

In [19]:
from jwst.scripts import asn_edit

In [19]:
asn_edit??

Type:        module
String form: <module 'jwst.scripts.asn_edit' from '/blue/adamginsburg/adamginsburg/repos/jwst/jwst/scripts/asn_edit.py'>
File:        /blue/adamginsburg/adamginsburg/repos/jwst/jwst/scripts/asn_edit.py
Source:     
#!/usr/bin/env python

import argparse

from jwst.associations import asn_edit


def main():
    """
    Parse command line, read, edit, write association file
    """

    # Parse command line arguments
    description_text = """
Edit Association File

This script adds or removes filenames from an association file. The
first argument is the name of the association file. Subsequent
arguments are the filenames. Options determine which operation is
performed: --add or --remove. If adding files the --type option sets
the exposure type for the new files. If removing file, the --ignore
option will not use the filename suffix when matching the filenames for
removal. Normally the output association file is the same as the input.
The --output option allows you to

In [20]:
shutil.copy('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn.json', 
            '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json')

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json'

In [22]:
asn_edit('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json')

TypeError: 'module' object is not callable

In [ ]:
#realign_to_vvv(filtername=filtername.lower(), fov_regname=fov_regname[regionname], basepath=basepath, module=module, fieldnumber=field,
#                                   imfile=realigned_vvv_filename, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15)

In [5]:
ims = glob(os.path.join(output_dir, f'*nrca*_destreak.fits'))
ims

['/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00003_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00004_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00003_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00001_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00004_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00003_nrcalong_destre

In [6]:
i = ims[0]
i1 = i.split('.')[0]+'_realigned.fits'
i1

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak_realigned.fits'

In [21]:
i

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00003_nrcalong_destreak.fits'

In [11]:
i1.split('_')[0][-3:]

'002'

In [18]:
i1

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak_realigned.fits'

In [7]:
realigned_vvv_i = i.split('.')[0]+'_realigned.fits'
shutil.copy(i, realigned_vvv_i)

raoffset = -1.65*u.arcsec
decoffset = -3.9*u.arcsec

realign_to_vvv(filtername=filtername.lower(), fov_regname=fov_regname[regionname], basepath=basepath, module=module, fieldnumber=field, 
               imfile=realigned_vvv_i, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15, raoffset=raoffset, decoffset=decoffset)

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:35:55.788508: Before realignment, offset is -0.0015155154187596054 arcsec, -0.018448119524805406 arcsec.  Found 269 matches.
2023-08-07T17:35:55.793358: CRVAL before 266.59541974652 -28.538974425935
2023-08-07T17:35:55.812019: CRVAL after 266.59587850083 -28.537885968124
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,


[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x2b6d6cac6ee0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b6d6a6dcbb0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b6d6a6f54c0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b6d6a6f5b80>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b6d6a6e3160>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b6d6a6fad00>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b6d6a6fa160>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b6d37eeaa90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2b6d6ca7b0a0>]

In [10]:
for i in ims:
    realigned_vvv_i = i.split('.')[0]+'_realigned.fits'
    shutil.copy(i, realigned_vvv_i)
    visit = i.split('_')[0][-3:]
    if visit == '001':
        raoffset = 0*u.arcsec
        decoffset = -8*u.arcsec
    elif visit == '002':
        raoffset = 0*u.arcsec#-1*u.arcsec
        decoffset = -4*u.arcsec
    else:
        raoffset = 0*u.arcsec
        decoffset = 0*u.arcsec
    realign_to_vvv(filtername=filtername.lower(), fov_regname=fov_regname[regionname], basepath=basepath, module=module, fieldnumber=field, 
                   imfile=realigned_vvv_i, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15, raoffset=raoffset, decoffset=decoffset)

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:10.185411: Before realignment, offset is -0.00253082204153543 arcsec, 0.016414905978479055 arcsec.  Found 296 matches.
2023-08-07T17:41:10.189610: CRVAL before 266.59541974652 -28.538974425935
2023-08-07T17:41:10.198148: CRVAL after 266.59542044953 -28.53786787452
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
    

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:11.600529: Before realignment, offset is 0.006203131192705769 arcsec, -0.03880539459686361 arcsec.  Found 231 matches.
2023-08-07T17:41:11.605020: CRVAL before 266.58849083407 -28.575359801581
2023-08-07T17:41:11.613688: CRVAL after 266.58848911098 -28.573126800083
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
   

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:13.311057: Before realignment, offset is -0.05898292281472095 arcsec, -0.037085173581630215 arcsec.  Found 198 matches.
2023-08-07T17:41:13.315934: CRVAL before 266.58378762432 -28.592813644458
2023-08-07T17:41:13.324575: CRVAL after 266.58380400847 -28.590581120799
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
  

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:14.805810: Before realignment, offset is -0.034158782375470764 arcsec, 0.0019842158827998446 arcsec.  Found 234 matches.
2023-08-07T17:41:14.810465: CRVAL before 266.59237008461 -28.557172894513
2023-08-07T17:41:14.819331: CRVAL after 266.59237957316 -28.556062334573
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
 

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:16.215254: Before realignment, offset is -0.008650702579870995 arcsec, -0.020027904672303976 arcsec.  Found 216 matches.
2023-08-07T17:41:16.219737: CRVAL before 266.58873993895 -28.574669560401
2023-08-07T17:41:16.228340: CRVAL after 266.58874234192 -28.572441774872
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
 

...

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:27.301131: Before realignment, offset is -0.05801487377539161 arcsec, -0.016078091984184084 arcsec.  Found 261 matches.
2023-08-07T17:41:27.305940: CRVAL before 266.59261881688 -28.556482695308
2023-08-07T17:41:27.314817: CRVAL after 266.59263493212 -28.55536711806
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
   

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:28.712895: Before realignment, offset is -0.015402021904264984 arcsec, -0.03960987450994935 arcsec.  Found 189 matches.
2023-08-07T17:41:28.717314: CRVAL before 266.58460781321 -28.593550093747
2023-08-07T17:41:28.725901: CRVAL after 266.58461209155 -28.591316868782
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
  

2023-08-07T17:41:29.023743: After re-realignment, offset is -0.023637323420189205 arcsec, -0.02555105036350369 arcsec using 189 matches
INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:30.364892: Before realignment, offset is 0.0035959925980932894 arcsec, 0.0021842864541099516 arcsec.  Found 257 matches.
2023-08-07T17:41:30.369843: CRVAL before 266.59154053673 -28.55716467436
2023-08-07T17:41:30.378238: CRVAL after 266.59153953784 -28.556054169995
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                theref

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:31.831025: Before realignment, offset is -0.0009048649189935531 arcsec, -0.026089722318545228 arcsec.  Found 193 matches.
2023-08-07T17:41:31.835620: CRVAL before 266.58377837707 -28.593542796145
2023-08-07T17:41:31.844384: CRVAL after 266.58377862842 -28.591313326778
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,


INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-07T17:41:33.876829: Before realignment, offset is 0.0014897899745847099 arcsec, -0.029909506461933688 arcsec.  Found 273 matches.
2023-08-07T17:41:34.469568: CRVAL before 266.59154983035 -28.556436704257
2023-08-07T17:41:34.492170: CRVAL after 266.59154941652 -28.55531728495
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
  